Этот блокнот позволяет поиграться с моделькой, классифицирующей тексты как токсичные или неполиткорректные. 

Всё самое весёлое - в последней ячейке. 

https://huggingface.co/cointegrated/rubert-tiny-toxicity

In [ ]:
!pip install transformers sentencepiece --quiet

     |████████████████████████████████| 2.6 MB 5.0 MB/s 
     |████████████████████████████████| 1.2 MB 43.0 MB/s 
     |████████████████████████████████| 636 kB 43.2 MB/s 
     |████████████████████████████████| 895 kB 29.2 MB/s 
     |████████████████████████████████| 3.3 MB 42.3 MB/s 


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
model_checkpoint = 'cointegrated/rubert-tiny-toxicity'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()

Downloading:   0%|          | 0.00/377 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/241k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/468k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/909 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/47.2M [00:00<?, ?B/s]

In [ ]:
def text2toxicity(text, aggregate=True):
    """ Calculate toxicity of a text (if aggregate=True) or a vector of toxicity aspects (if aggregate=False)"""
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()
    if isinstance(text, str):
        proba = proba[0]
    if aggregate:
        return 1 - proba.T[0] * (1 - proba.T[-1])
    return proba

print(text2toxicity('я люблю нигеров', True))

print(text2toxicity('я люблю нигеров', False))

print(text2toxicity(['я люблю нигеров', 'я люблю африканцев'], True))

print(text2toxicity(['я люблю нигеров', 'я люблю африканцев'], False))

0.9350118728093193
[0.9715758  0.0180863  0.0045551  0.00189755 0.9331106 ]
[0.93501186 0.04156357]
[[9.7157580e-01 1.8086294e-02 4.5550885e-03 1.8975559e-03 9.3311059e-01]
 [9.9979788e-01 1.9048342e-04 1.5297388e-04 1.7452303e-04 4.1369814e-02]]


In [ ]:
%%time
print(text2toxicity('Иди ты нафиг!'))

0.4770178304282737
CPU times: user 9.63 ms, sys: 0 ns, total: 9.63 ms
Wall time: 12.2 ms
